# Step5.1: get_boxes


运行它获得boxes裁剪区块数据，为生成文件夹做准备

### 1. Setup

* First, set up Python, `numpy`, and `matplotlib`.

In [1]:
# set up Python environment: numpy for numerical routines, and matplotlib for plotting
import numpy as np
import matplotlib.pyplot as plt
import pylab
from skimage import transform
# display plots in this notebook
%matplotlib inline

import os
os.environ["KMP_DUPLICATE_LIB_OK"]="TRUE"
# 防止挂掉
os.environ['GLOG_minloglevel'] = '3' 
# 忽略caffe打印的结构信息！要写在import caffe之前！因为在导入caffe时caffe会加载GLOG！

# set display defaults
plt.rcParams['figure.figsize'] = (12, 9)        # small images
plt.rcParams['image.interpolation'] = 'nearest'  # don't interpolate: show square pixels
plt.rcParams['image.cmap'] = 'gray'  # use grayscale output rather than a (potentially misleading) color heatmap
print("ok")

ok


In [2]:
pwd

'/root/hujunyao/ACL22-sarcasm-code'

* Load `caffe`.

In [3]:
# Change dir to caffe root or prototxt database paths won't work wrong
import os
os.chdir('/root/hujunyao/ACL22-sarcasm-code/bottom-up-attention')
!pwd

/root/hujunyao/ACL22-sarcasm-code/bottom-up-attention


In [4]:
# The caffe module needs to be on the Python path;
#  we'll add it here explicitly.
import sys
sys.path.insert(0, './caffe/python/')
sys.path.insert(0, './lib/')
sys.path.insert(0, './tools/')
import caffe

In [5]:
caffe.set_mode_gpu()
caffe.set_device(0)

In [6]:
data_path = './data/genome/1600-400-20'

# Load classes
classes = ['__background__']
with open(os.path.join(data_path, 'objects_vocab.txt')) as f:
    for object in f.readlines():
        classes.append(object.split(',')[0].lower().strip())

# Load attributes
attributes = ['__no_attribute__']
with open(os.path.join(data_path, 'attributes_vocab.txt')) as f:
    for att in f.readlines():
        attributes.append(att.split(',')[0].lower().strip())

In [7]:
# Check object extraction
from fast_rcnn.config import cfg, cfg_from_file
from fast_rcnn.test import im_detect,_get_blobs
from fast_rcnn.nms_wrapper import nms
import cv2

# GPU_ID = 0   # if we have multiple GPUs, pick one 
caffe.set_device(0)  
caffe.set_mode_gpu()
net = None
cfg_from_file('experiments/cfgs/faster_rcnn_end2end_resnet.yml')

weights = './data/resnet101_faster_rcnn_final.caffemodel'
prototxt = 'models/vg/ResNet-101/faster_rcnn_end2end_final/test.prototxt'

In [8]:
net = caffe.Net(prototxt, weights, caffe.TEST)
print("done")

done


In [9]:
import os
imgs = os.listdir("../data/dataset_image")
imgs.sort()
imgs[:10]
print(len(imgs))

24635


In [ ]:
import pickle as pk
from tqdm import tqdm
dic = {}
# imgs = imgs[:1000]
for idx in tqdm(range(len(imgs))):
    img = imgs[idx]
    im_file = "../data/dataset_image/" + img
    conf_thresh=0.4
    min_boxes=10
    max_boxes=20

    im = cv2.imread(im_file)
    
    b,g,r = cv2.split(im)#分别提取B、G、R通道
    im = cv2.merge([r,g,b])
    scores, boxes, attr_scores, rel_scores = im_detect(net, im)

    # Keep the original boxes, don't worry about the regression bbox outputs
    rois = net.blobs['rois'].data.copy()
    # unscale back to raw image space
    blobs, im_scales = _get_blobs(im, None)

    cls_boxes = rois[:, 1:5] / im_scales[0]
    cls_prob = net.blobs['cls_prob'].data
    attr_prob = net.blobs['attr_prob'].data
    pool5 = net.blobs['pool5_flat'].data

    # Keep only the best detections
    max_conf = np.zeros((rois.shape[0]))
    for cls_ind in range(1,cls_prob.shape[1]):
        cls_scores = scores[:, cls_ind]
        dets = np.hstack((cls_boxes, cls_scores[:, np.newaxis])).astype(np.float32)
        keep = np.array(nms(dets, cfg.TEST.NMS))
        max_conf[keep] = np.where(cls_scores[keep] > max_conf[keep], cls_scores[keep], max_conf[keep])

    keep_boxes = np.where(max_conf >= conf_thresh)[0]
    if len(keep_boxes) < min_boxes:
        keep_boxes = np.argsort(max_conf)[::-1][:min_boxes]
    elif len(keep_boxes) > max_boxes:
        keep_boxes = np.argsort(max_conf)[::-1][:max_boxes]
    ############################

    im = cv2.cvtColor(im, cv2.COLOR_BGR2RGB)
    boxes = cls_boxes[keep_boxes]
    objects = np.argmax(cls_prob[keep_boxes][:,1:], axis=1)
    attr_thresh = 0.1
    attr = np.argmax(attr_prob[keep_boxes][:,1:], axis=1)
    attr_conf = np.max(attr_prob[keep_boxes][:,1:], axis=1)
    
    dic[img] = list()
    
    for i in range(len(keep_boxes)):
        bbox = boxes[i]
        if bbox[0] == 0:
            bbox[0] = 1
        if bbox[1] == 0:
            bbox[1] = 1
        cls = classes[objects[i]+1]
        if attr_conf[i] > attr_thresh:
            # 小小吐槽，这里应该把" "改成"_"!,不然后面坑死，不过在5.2已经进行了补救，时间不够了,就不重新生成box了！！！！
            cls = attributes[attr[i]+1] + "_" + cls  
            
        dic[img].append([bbox,cls])
pk.dump(dic,open("boxes.pkl",'wb'))

 16%|█▌        | 3951/24635 [45:47<4:01:36,  1.43it/s]

### `boxes.pkl`的保存格式

对于位置列表：`（左上x,左上y,右下x,右下y)`

```
os.chdir('/root/hujunyao/ACL22-sarcasm-code/bottom-up-attention'){'893019140839145472.jpg': [
  [array([3.3292169e+02, 1.0000000e+00, 1.1991466e+03, 3.4800439e+02],dtype=float32),'gray bridge'],
  [array([ 248.29492,  175.57552, 1074.2344 ,  894.50665], dtype=float32),'gray sky'],
  [array([ 107.84516,  679.2003 , 1198.2689 ,  894.50665], dtype=float32),'green trees'],
  [array([1060.3639 ,  633.52344, 1199.1466 ,  894.50665], dtype=float32),'green tree'],
  ...
  ],
 '818606588231553024.jpg': [],...
}
```

In [ ]:
print(classes)

In [ ]:
print(attributes)